# 모달 주요도 추출 예제 코드
- 101번 사용자에 대한 예제 코드
- 101_tension_MIL_Model Train code.ipynb에서 학습된 모델 사용

# 필요 라이브러리 로드

In [1]:
%cd /workspace

import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from human_lifelog_mil_pytorch.FocalLoss import FocalLoss
from human_lifelog_mil_pytorch.human_lifelog_mil_pytorch import MILNet
from human_lifelog_mil_pytorch.human_lifelog_mil_pytorch import HumanDataset
from human_lifelog_mil_pytorch.utils import getF1Score

import torch
import os
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings( 'ignore' )

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook

device = torch.device("cuda:0")

/workspace


# 데이터 로드
- 101_tension_MIL_Model Train code.ipynb와 동일한 방식으로 데이터 로드
- 모달 주요도는 testset에서 추출 예정

In [2]:
user = '101'

In [3]:
df = pd.read_pickle('/workspace/Data analysis (zaesoek)/2019/'+user+'_prpDf.pickle')

df = df.drop(df[df['e4AccLen']!=1920].index, 0)
df = df.drop(df[df['e4BvpLen']!=3840].index, 0)
df = df.drop(df[df['e4EdaLen']!=240].index, 0)
df = df.drop(df[df['e4TempLen']!=240].index, 0)

df = df.drop(df[df['mAccLen']<1786].index, 0)
df = df.drop(df[df['mGyrLen']<1786].index, 0)
df = df.drop(df[df['mMagLen']<1786].index, 0)

encoder = LabelEncoder()
df['action_label'] = encoder.fit_transform(df['action'])
encoder = LabelEncoder()
df['positive_label'] = encoder.fit_transform(df['emotionPositive'])
encoder = LabelEncoder()
df['tension_label'] = encoder.fit_transform(df['emotionTension'])

for i in df.index:
    df['e4Hr'][i] = np.mean(df['e4Hr'][i])
    df['mGps'][i] = np.mean(df['mGps'][i])
    
    if df['mAccLen'][i] == 1787:
        df['mAcc'][i] = df['mAcc'][i][:1786]
    if df['mGyrLen'][i] == 1787:
        df['mGyr'][i] = df['mGyr'][i][:1786]
    if df['mMagLen'][i] == 1787:
        df['mMag'][i] = df['mMag'][i][:1786]

df.head()

,userName,emotionPositive,emotionTension,action,gender,height,weight,e4Acc,e4Bvp,e4Eda,...,e4EdaLen,e4HrLen,e4TempLen,mAccLen,mGpsLen,mGyrLen,mMagLen,action_label,positive_label,tension_label
ts,,,,,,,,,,,,,,,,,,,,,
1578184440,101,6,2,recreation_etc,F,161.0,52.0,"[[0.9375, -0.25, -0.03125], [0.9375, -0.25, -0...","[[-0.5897236], [-2.0923119], [-3.8966255], [-5...","[[0.029454641], [0.029454641], [0.032015916], ...",...,240,32,240,1786,1,1786,1786,2,2,1
1578184560,101,6,2,recreation_etc,F,161.0,52.0,"[[0.9375, -0.21875, -0.0625], [0.9375, -0.2187...","[[-14.854795], [-17.59889], [-20.453917], [-22...","[[0.070435025], [0.0729963], [0.0729963], [0.0...",...,240,6,240,1786,1,1787,1787,2,2,1
1578184620,101,6,2,recreation_etc,F,161.0,52.0,"[[0.921875, -0.21875, -0.140625], [0.921875, -...","[[-5.4048615], [-6.8878326], [-9.298111], [-12...","[[0.08708331], [0.083241396], [0.085802674], [...",...,240,3,240,1786,1,1786,1786,2,2,1
1578184920,101,6,2,recreation_etc,F,161.0,52.0,"[[-0.28125, -0.578125, 0.734375], [-0.28125, -...","[[10.687408], [13.796631], [16.10376], [16.737...","[[0.15111516], [0.15367644], [0.1562377], [0.1...",...,240,8,240,1786,1,1786,1786,2,2,1
1578185040,101,6,2,recreation_etc,F,161.0,52.0,"[[-0.6875, -0.40625, 0.578125], [-0.6875, -0.4...","[[-5.266464], [-8.345547], [-11.21323], [-13.8...","[[0.13318624], [0.13446687], [0.13574752], [0....",...,240,64,240,1786,1,1787,1787,2,2,1


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, shuffle = True, random_state=32)
train_df, valid_df = train_test_split(train_df, test_size=0.125, shuffle = True, random_state=32)

print(len(train_df), len(valid_df), len(test_df))

trainset = HumanDataset(train_df)
validset = HumanDataset(valid_df)
testset = HumanDataset(test_df)

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=512, num_workers=0, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(validset, batch_size=32, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(testset, batch_size=32, num_workers=0)

1290 185 369


# Tension model 로드
- 101_tension_MIL_Model Train code.ipynb에서 학습된 모델 로드

In [5]:
tension_milnet = MILNet(class_num = len(df['tension_label'].unique())).to(device)
best_loss_path = '/workspace/MILs(zaeseok)/2019 users/tension models/'+user+'.p'
tension_milnet.load_state_dict(torch.load(best_loss_path))

<All keys matched successfully>

# Testset에서 모달 주요도 추출

In [6]:
tension_atts = []

tension_milnet.eval()
for batch_id, (
    e4Acc, e4Bvp, e4Eda, e4Hr, e4Temp,
    mAcc, mGps, mGyr, mMag,
    emotionPositive, emotionTension, action
) in enumerate(test_dataloader):
    e4Acc = e4Acc.to(device)
    e4Bvp = e4Bvp.to(device)
    e4Eda = e4Eda.to(device)
    e4Hr = e4Hr.float().to(device)
    e4Temp = e4Temp.to(device)
    mAcc = mAcc.to(device)
    mGps = mGps.float().to(device)
    mGyr = mGyr.to(device)
    mMag = mMag.to(device)
    label = emotionTension.long().to(device)
    
    # att을 통해 각 모달의 주요도 추출
    _, _, _, att = tension_milnet(e4Acc, e4Bvp, e4Eda, e4Hr, e4Temp, mAcc, mGps, mGyr, mMag)

    temp_atts = att.detach().cpu().numpy()
    tension_atts += list(temp_atts)

tension_atts = np.array(tension_atts).reshape(-1, 9)

with open('/workspace/MILs(zaeseok)/2019 users/tension_atts/'+user+'.p', 'wb') as f:
    pickle.dump(tension_atts, f, pickle.HIGHEST_PROTOCOL)

# 모달 주요도 형태
- 각 데이터에 대하여 모달 별 주요도가 추출 (모달은 9개이므로 9개에 대한 주요도 추출)

In [7]:
tension_atts

array([[0.05016539, 0.02071478, 0.08433771, ..., 0.22114296, 0.09544072,
        0.10374151],
       [0.04556508, 0.01263833, 0.10977403, ..., 0.22408846, 0.06326192,
        0.11171645],
       [0.10095838, 0.01394055, 0.11276968, ..., 0.1867308 , 0.10661411,
        0.06216093],
       ...,
       [0.06113615, 0.0162528 , 0.09315126, ..., 0.22908539, 0.06791632,
        0.05998908],
       [0.09962416, 0.01594216, 0.10637611, ..., 0.18703876, 0.11550146,
        0.04581542],
       [0.09384847, 0.02099067, 0.11735202, ..., 0.20181094, 0.06687717,
        0.06284719]], dtype=float32)

In [8]:
tension_atts.shape

(369, 9)